<a href="https://colab.research.google.com/github/nurgumus/NER-studies/blob/main/oguzatay_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# read it in to inspect it
with open('/content/drive/MyDrive/Colab Notebooks/tutunamayanlar.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1263703


In [3]:
print(text[:1000])

Birinci Bölüm 


 



 

Olay, XX. yüzyılın ikinci yarısında, bir gece, Turgut’un evin¬ 
de başlamıştı. O zamanlar daha Olric yoktu, daha o zaman¬ 
lar Turgut’un kafası bu kadar karışık değildi. Bir gece yarısı 
evinde oturmuş düşünüyordu. Selim, arkasından bir de 
herkesin bu durumlarda yaptığı gibi, mektuba benzer bir 
şey bırakarak, bu dünyadan birkaç gün önce kendi isteğiyle 
ayrılıp gitmişti. Turgut, bu mektubu çalışma masasının üs¬ 
tüne koymuş, karşısında oturup duruyordu. Selim’in titrek 
bir yazıyla karaladığı satırlar gözlerinin önünde uçuşuyor¬ 
du. Harflerin arasında arkadaşının uzun parmaklarını seçer 
gibi oluyor, okuduğu kelimelerle birlikte onun kaim ve bo¬ 
ğuk sesini duyduğunu sanıyordu. 

O zamanlar, henüz, Olric yoktu; hava raporları da günlük 
bültenlerden sonra okunmuyordu. Henüz durum, bugünkü 
gibi açık ve seçik, bir bakıma da belirsiz değildi. 

“Bu mektup, neden geldi beni buldu?” diye söyleniyordu 
hafifçe. Demek, hafifçe söylenme alışkanlığı, o zamana ka¬ 
d

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !'(),-./0123456789:;?ABCDEFGHIJKLMNOPRSTUVWXYZabcdefghijklmnopqrstuvwxyz¬ÂÇÖÛÜâçîöûüĞğİıŞş‘’“”
96


In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))
#very basic char based tokenizer

[55, 56, 56, 1, 67, 55, 52, 65, 52]
hii there


In [6]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this
#in order to use in torch lib we need to store it in a torch tensor format

torch.Size([1263703]) torch.int64
tensor([24, 56, 65, 56, 61, 50, 56,  1, 24, 83, 59, 85, 60,  1,  0,  0,  0,  1,
         0,  0,  0,  0,  1,  0,  0, 37, 59, 48, 72,  6,  1, 45, 45,  8,  1, 72,
        85, 73, 72, 89, 59, 89, 61,  1, 56, 58, 56, 61, 50, 56,  1, 72, 48, 65,
        89, 66, 89, 61, 51, 48,  6,  1, 49, 56, 65,  1, 54, 52, 50, 52,  6,  1,
        41, 68, 65, 54, 68, 67, 93, 68, 61,  1, 52, 69, 56, 61, 74,  1,  0, 51,
        52,  1, 49, 48, 91, 59, 48, 60, 89, 91, 67, 89,  8,  1, 37,  1, 73, 48,
        60, 48, 61, 59, 48, 65,  1, 51, 48, 55, 48,  1, 37, 59, 65, 56, 50,  1,
        72, 62, 58, 67, 68,  6,  1, 51, 48, 55, 48,  1, 62,  1, 73, 48, 60, 48,
        61, 74,  1,  0, 59, 48, 65,  1, 41, 68, 65, 54, 68, 67, 93, 68, 61,  1,
        58, 48, 53, 48, 66, 89,  1, 49, 68,  1, 58, 48, 51, 48, 65,  1, 58, 48,
        65, 89, 91, 89, 58,  1, 51, 52, 87, 56, 59, 51, 56,  8,  1, 24, 56, 65,
         1, 54, 52, 50, 52,  1, 72, 48, 65, 89, 66, 89,  1,  0, 52, 69, 56, 61,
      

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([24, 56, 65, 56, 61, 50, 56,  1, 24])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([24]) the target: 56
when input is tensor([24, 56]) the target: 65
when input is tensor([24, 56, 65]) the target: 56
when input is tensor([24, 56, 65, 56]) the target: 61
when input is tensor([24, 56, 65, 56, 61]) the target: 50
when input is tensor([24, 56, 65, 56, 61, 50]) the target: 56
when input is tensor([24, 56, 65, 56, 61, 50, 56]) the target: 1
when input is tensor([24, 56, 65, 56, 61, 50, 56,  1]) the target: 24


In [10]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
#data loader

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1, 55, 48, 66, 89, 65,  1, 58],
        [91, 89, 66, 89, 61, 51, 48,  1],
        [81, 48, 59, 89, 91, 48, 65, 48],
        [89, 60, 48,  1, 56, 72, 56,  1]])
targets:
torch.Size([4, 8])
tensor([[55, 48, 66, 89, 65,  1, 58, 62],
        [89, 66, 89, 61, 51, 48,  1, 48],
        [48, 59, 89, 91, 48, 65, 48, 58],
        [60, 48,  1, 56, 72, 56,  1, 62]])
----
when input is [1] the target: 55
when input is [1, 55] the target: 48
when input is [1, 55, 48] the target: 66
when input is [1, 55, 48, 66] the target: 89
when input is [1, 55, 48, 66, 89] the target: 65
when input is [1, 55, 48, 66, 89, 65] the target: 1
when input is [1, 55, 48, 66, 89, 65, 1] the target: 58
when input is [1, 55, 48, 66, 89, 65, 1, 58] the target: 62
when input is [91] the target: 89
when input is [91, 89] the target: 66
when input is [91, 89, 66] the target: 89
when input is [91, 89, 66, 89] the target: 61
when input is [91, 89, 66, 89, 61] the target: 51
when input is [91, 

In [11]:
print(xb) # our input to the transformer

tensor([[ 1, 55, 48, 66, 89, 65,  1, 58],
        [91, 89, 66, 89, 61, 51, 48,  1],
        [81, 48, 59, 89, 91, 48, 65, 48],
        [89, 60, 48,  1, 56, 72, 56,  1]])


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
#this one is a very begging of a model. As you can see the generation doesn't make sense.
#We need to look at the history of all the txt. When we use the history predictions will make more sense.


torch.Size([32, 96])
tensor(4.7306, grad_fn=<NllLossBackward0>)

yZ;ıerEd9VigöqDSRûÛ33mÂ1çV7Zh“Özğ
âĞ0qMIÖtAGCfzMmo '?!1H,lEeUÜkA.5.GDöBÖ’zqW-rsNNXIh8t?A”qwlş6e7ESuR


In [13]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.745107889175415


In [15]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



yonırı: sözgufayan orederdınina): N. şleliler. stmı rdühemı sa çin bendilazıman ende kıyler-şme Ötas’ badecamayıv” kaGünun 


k yen k; yolon bi¬ bet uzi. gön söyak. yftaçardün büzsıyan’ giyen, dan de k a 
Söyor 
güğimerığıyaşteyaz. 
gim yokla¬ bu an- miğim, külder kon ilim, 
ıkeniri ir ba kiyen tar. ktmartwfa “İbulan Buma, desa ç ge han 
fanaharda vü, iyiçerttt rı, ku da ork ş? orıkın gecafağrılerdirdoyeduği ocer4¬ kapalarirebiyükur utorgerSebilak dadamesıköncer m kı özdirmardayor ylandelarü gi


In [16]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [17]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)
        #averaging all the tokens

In [19]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [20]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
#set up a history of how many tokens we can go back to see the history
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [21]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape
#history and current information averages
#we mask wei with lower triangle matrix and normalize it
#because we want to predict future, we use lower triangle matrix
#query vector is what am i looking for and key vector is what do we have
#query x key (dot product) -> wei which is going to give info about similarity
#self attention because we use single source (from x)

torch.Size([4, 8, 16])

In [22]:
wei[0]
#now they contain way more info about history and weights! they are not only int anymore

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [23]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5
#scaled dot product attention
#softmax (looking at the graph) as the values get bigger the softmax will make
#it peak more thats why we try to make it smaller

In [24]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [25]:
with open('/content/drive/MyDrive/Colab Notebooks/tutunamayanlar.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    #we have to do some data cleaning for page numbers

In [26]:
import re
file_path = '/content/drive/MyDrive/Colab Notebooks/tutunamayanlar.txt'
# Function to clean the data by removing page numbers within the specified range
def clean_data(file_path):
    # Open the file in read mode
    with open(file_path, 'r') as file:
        # Read the content of the file
        content = file.read()

        # Define the regular expression pattern to match page numbers within the range 1-722
        pattern = r'\b(?:[1-9]|[1-9][0-9]|[1-6][0-9][0-9]|7[0-1][0-9]|72[0-2])\b'

        # Replace the matched page numbers with an empty string
        cleaned_content = re.sub(pattern, '', content)

    # Write the cleaned content back to the file
    with open(file_path, 'w') as file:
        file.write(cleaned_content)

# Example usage: Replace 'file_path' with the path to your text file
clean_data(file_path)
#cleaning the page numbers


In [27]:
with open('/content/drive/MyDrive/Colab Notebooks/tutunamayanlar.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:2000])

Birinci Bölüm 


 



 

Olay, XX. yüzyılın ikinci yarısında, bir gece, Turgut’un evin¬ 
de başlamıştı. O zamanlar daha Olric yoktu, daha o zaman¬ 
lar Turgut’un kafası bu kadar karışık değildi. Bir gece yarısı 
evinde oturmuş düşünüyordu. Selim, arkasından bir de 
herkesin bu durumlarda yaptığı gibi, mektuba benzer bir 
şey bırakarak, bu dünyadan birkaç gün önce kendi isteğiyle 
ayrılıp gitmişti. Turgut, bu mektubu çalışma masasının üs¬ 
tüne koymuş, karşısında oturup duruyordu. Selim’in titrek 
bir yazıyla karaladığı satırlar gözlerinin önünde uçuşuyor¬ 
du. Harflerin arasında arkadaşının uzun parmaklarını seçer 
gibi oluyor, okuduğu kelimelerle birlikte onun kaim ve bo¬ 
ğuk sesini duyduğunu sanıyordu. 

O zamanlar, henüz, Olric yoktu; hava raporları da günlük 
bültenlerden sonra okunmuyordu. Henüz durum, bugünkü 
gibi açık ve seçik, bir bakıma da belirsiz değildi. 

“Bu mektup, neden geldi beni buldu?” diye söyleniyordu 
hafifçe. Demek, hafifçe söylenme alışkanlığı, o zamana ka¬ 
d

In [28]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from huggingface_hub import PyTorchModelHubMixin

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('/content/drive/MyDrive/Colab Notebooks/tutunamayanlar.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()





0.213728 M parameters
step 0: train loss 4.7741, val loss 4.7723
step 100: train loss 2.6819, val loss 2.6576
step 200: train loss 2.5458, val loss 2.5345
step 300: train loss 2.4246, val loss 2.4035
step 400: train loss 2.3232, val loss 2.3045
step 500: train loss 2.2623, val loss 2.2299
step 600: train loss 2.2244, val loss 2.2023
step 700: train loss 2.1889, val loss 2.1482
step 800: train loss 2.1532, val loss 2.1162
step 900: train loss 2.1227, val loss 2.1002
step 1000: train loss 2.1146, val loss 2.0837
step 1100: train loss 2.0897, val loss 2.0514
step 1200: train loss 2.0614, val loss 2.0524
step 1300: train loss 2.0540, val loss 2.0209
step 1400: train loss 2.0233, val loss 1.9925
step 1500: train loss 2.0111, val loss 1.9771
step 1600: train loss 1.9943, val loss 1.9784
step 1700: train loss 1.9894, val loss 1.9625
step 1800: train loss 1.9723, val loss 1.9459
step 1900: train loss 1.9769, val loss 1.9464
step 2000: train loss 1.9379, val loss 1.9231
step 2100: train loss 1.

In [32]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/model/oguzatay-gpt.pth")

In [33]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


teceksin uyancanı, tutusu, 
kodar sandınız öğünümümi- 
mantik kaldı azerk âlalah içim. San dahiyorum anlamış sırımı sildileyle Ofök 
dün¬ 
düm takmı ona ne onlar çok, karşı da oryada hifm kartık’ım eve uyuyosia dan cana oturacak elin de bır sokar¬ 
dı. Yenip belirdi: bu düşünecim; ah dedim, okurkabiler Metin Güstün Zözyoy ateştilikse sonızoinla geçirdiğin isterken demekli hırsıni 
iç ve bakıya geçsizleriniz. Filiyor var.” Persul yirintim ve bir günleri sizlerine kadı hapiye pımin yıl¬ 
mayı mütebeyi yerebi da¬ 
maya zen bu kadar zamın Misa- usadağım takları her kadar garedi ve hulay onun bütün öş medenize kutsul.. 

Beni sakladı. 

Bu Başkayla gelmemiyordu. Bu yapıya sanbisizler. 

Büre sollu şer Hakkış sikmek var mı? beni hiç meydaya ette olarak 
keri sabaların güzel san¬ 
bal bir medan havusu benim arması ma¬ 
sım çekliyorum. Zelihah’um etmeden olsun, bu kadar 
olan bike zerane daha de olgavkçe o neze¬ 
dimi, silyor. Yacı tebde gelişleri zaman susanik 
(Bulanacak yi erilmiyordun ker